# Forecast Reconciliation from Scratch in Python
Reconcile hierarchal forecasts into coherent forecasts using Python.

## Introduction
In my experience, it is somewhat unusual to work with real time-series data that does not have underlying "levels". For example, consider the data that might be produced via transactions at a grocery store. Transactions can be described at an individual product level, a shopper level, or at a store level. The products purchased might also be categorized into a specific type of product, and those categories might in turn fall into broader categories. For a business owner, this complex hierarchal structure can complicate making accurate and unbiased forecasts for their business. A data-driven person will most likely produce forecasts for each of the different levels, but they don't always add up. This is where reconciliation becomes necessary.

The majority of my explanations throughout this post are based on the book [*Forecasting: Principles and Practice*](https://otexts.com/fpp3/) by Rob J. Hyndman and George Athanasopoulos - an excellent resource for forecasting in general and completely free online. I highly recommend that you spend some time reading Hyndman's [more detailed explanation](https://otexts.com/fpp3/hierarchical.html) of reconciling hierarchal and grouped forecasts at some point. It is not my intention here to replace this book, although I still intend to give a comprehensive explanation of forecasting reconciliation. My intention in writing this blog-post is to provide whomever is reading this with the background needed to develop their own forecast reconciliation code. I have found that the [existing tools]() I have used are not (in my opinion) at a stage in their development where they can be considered reliable in a production environement, or cannot be generalized to every scenario (I know this from personal experience). I have also found that staring at a mathematical formula or applying some pre-defined function that somebody else created are not the best ways to gain a firm understanding of what is actually happening. Hopefully by sharing some short explanations of how one might go about writing their own Python code, I can also provide some additional insight that might not be attained otherwise.

## Hierarchal Time Series
To begin, I will give a brief introduction to the differences between hierarchal and grouped time series (no, they are not quite the same thing). A time series is considered *hierarchal*
when lower levels withing the hierarchy only fall under one domain. For example consider the following tree:

In [1]:
from treelib import Tree

tree = Tree()
tree.create_node("Total", "total")  # root node
tree.create_node("Cat 1", "cat1", parent="total")
tree.create_node("Cat 2", "cat2", parent="total")
tree.create_node("SubCategory 1", "sub1", parent="cat1")
tree.create_node("SubCategory 2", "sub2", parent="cat1")
tree.create_node("SubCategory 3", "sub3", parent="cat2")
tree.create_node("SubCategory 4", "sub4", parent="cat2")
tree.show()

Total
├── Cat 1
│   ├── SubCategory 1
│   └── SubCategory 2
└── Cat 2
    ├── SubCategory 3
    └── SubCategory 4



`Sub-Category 1` falls exclusively under `Category 1`, `Sub-Category 4` falls exlusively under `Sub-Category 2`, etc. This *exclusivity* is what defines a hierarchal time series. It is also important to note that mathematically, each of the sub-categories should add up to the category above them, and `Total` is also the total sum of each of the bottom-level categories. A mathematical expression might look something like this:

*T* = *C1* + *C2* = *SC1* + *SC2* + *SC3* + *SC4* <br>
*C1* = *SC1* + *SC2* <br>
*C2* = *SC3* + *SC4* <br>

A *grouped* time series is when that exclusivity between sub-domains does not exist. For example, consider the same hierarchy tree but with only three sub-categories spread across each of the primary categories:

In [2]:
tree = Tree()
tree.create_node("Total", "total")  # root node
tree.create_node("Category 1", "cat1", parent="total")
tree.create_node("Category 2", "cat2", parent="total")
tree.create_node("SubCategory 1", "sub1", parent="cat1")
tree.create_node("SubCategory 2", "sub2", parent="cat1")
tree.create_node("SubCategory 3", "sub3", parent="cat1")
tree.create_node("SubCategory 1", "sub4", parent="cat2")
tree.create_node("SubCategory 2", "sub5", parent="cat2")
tree.create_node("SubCategory 3", "sub6", parent="cat2")
tree.show()

Total
├── Category 1
│   ├── SubCategory 1
│   ├── SubCategory 2
│   └── SubCategory 3
└── Category 2
    ├── SubCategory 1
    ├── SubCategory 2
    └── SubCategory 3



The same hierarchy could also be described by the following tree:

In [3]:
tree = Tree()
tree.create_node("Total", "total")  # root node
tree.create_node("SubCategory 1", "sub1", parent="total")
tree.create_node("SubCategory 2", "sub2", parent="total")
tree.create_node("SubCategory 3", "sub3", parent="total")
tree.create_node("Category 1", "cat1", parent="sub1")
tree.create_node("Category 2", "cat2", parent="sub1")
tree.create_node("Category 1", "cat3", parent="sub2")
tree.create_node("Category 2", "cat4", parent="sub2")
tree.create_node("Category 1", "cat5", parent="sub3")
tree.create_node("Category 2", "cat6", parent="sub3")
tree.show()

Total
├── SubCategory 1
│   ├── Category 1
│   └── Category 2
├── SubCategory 2
│   ├── Category 1
│   └── Category 2
└── SubCategory 3
    ├── Category 1
    └── Category 2



This complexity of multiple arangements of the hierarchy groups means that the original formulas used are no longer valid. Hyndman [describes](https://otexts.com/fpp3/hts.html#grouped-time-series) this structural concept as *"not naturally disaggregat[ing] in a unique hierarchical manner."* 

The differences between hierarchal and grouped time series are important to understand, because the summing matrix (explained briefly in the next section) is dependent on the structure of the time series.

## Building-Blocks of Coherent Forecasts
Coherent - or *reconciled* - forecasts are constructed from a few key components:

**Base Forecasts:** <br>
Forecasts at each hierarchal level represented as an *m* x *n* matrix (*m* rows and *n* columns), where the columns of the matrix represent the hierarchal levels of the data and the rows represent each time period of the forecast horizon.

**Summing matrix:** <br>
Describes the hierarchal/grouped structure of the data. For hierarchal summing matrices, the number of columns matches the number of unique categories on the bottom level. The number of rows is determined by the total number of unique categories across all levels. The values of the summing matrix are binary values that represent which bottom-level (column) categories map to each of the hierarchies across all levels. Consider the example shared previously of hierarchal data with two categories and four sub-categories. The summing matrix would look like the following:

In [4]:
import pandas as pd
pd.DataFrame(
    data = {
        'Sub-Category 1': [1,1,0,1,0,0,0],
        'Sub-Category 2': [1,1,0,0,1,0,0],
        'Sub-Category 3': [1,0,1,0,0,1,0],
        'Sub-Category 4': [1,0,1,0,0,0,1]
        },
    index = [
        'Total', 'Category 1', 'Category 2', 
        'Category 1 -> Sub-Category 1', 
        'Category 1 -> Sub-Category 2',
        'Category 2 -> Sub-Category 3',
        'Category 2 -> Sub-Category 4'
        ])

,Sub-Category 1,Sub-Category 2,Sub-Category 3,Sub-Category 4
Total,1,1,1,1
Category 1,1,1,0,0
Category 2,0,0,1,1
Category 1 -> Sub-Category 1,1,0,0,0
Category 1 -> Sub-Category 2,0,1,0,0
Category 2 -> Sub-Category 3,0,0,1,0
Category 2 -> Sub-Category 4,0,0,0,1


Because grouped time series data does not disaggregate in a "unique manner," there is not one unique bottom level. This means that the summing matrix needs additional columns as well as additional rows. The grouped data from earlier in the post would look like this:

In [5]:
pd.DataFrame(
    data = {
        'Cat 1 / Sub-Cat 1': [1,1,0,1,0,0,1,0,0,0,0,0], 
        'Cat 1 / Sub-Cat 2': [1,1,0,0,1,0,0,1,0,0,0,0],
        'Cat 1 / Sub-Cat 3': [1,1,0,0,0,1,0,0,1,0,0,0],
        'Cat 2 / Sub-Cat 1': [1,0,1,1,0,0,0,0,0,1,0,0],
        'Cat 2 / Sub-Cat 2': [1,0,1,0,1,0,0,0,0,0,1,0],
        'Cat 2 / Sub-Cat 3': [1,0,1,0,0,1,0,0,0,0,0,1]
        },
    index = [
        'Total', 
        'Category 1', 
        'Category 2', 
        'Sub-Category 1',
        'Sub-Category 2',
        'Sub-Category 3',
        'Cat 1 / Sub-Cat 1', 
        'Cat 1 / Sub-Cat 2',
        'Cat 1 / Sub-Cat 3',
        'Cat 2 / Sub-Cat 1',
        'Cat 2 / Sub-Cat 2',
        'Cat 2 / Sub-Cat 3'
        ])

,Cat 1 / Sub-Cat 1,Cat 1 / Sub-Cat 2,Cat 1 / Sub-Cat 3,Cat 2 / Sub-Cat 1,Cat 2 / Sub-Cat 2,Cat 2 / Sub-Cat 3
Total,1,1,1,1,1,1
Category 1,1,1,1,0,0,0
Category 2,0,0,0,1,1,1
Sub-Category 1,1,0,0,1,0,0
Sub-Category 2,0,1,0,0,1,0
Sub-Category 3,0,0,1,0,0,1
Cat 1 / Sub-Cat 1,1,0,0,0,0,0
Cat 1 / Sub-Cat 2,0,1,0,0,0,0
Cat 1 / Sub-Cat 3,0,0,1,0,0,0
Cat 2 / Sub-Cat 1,0,0,0,1,0,0


**Mapping Matrix:** <br>
 The key componenent of forecast reconciliation is the mapping matrix. This matrix varies depending on the reconciliation method used, but the principle remains the same. When you multiply the mapping matrix with the summing and base forecast matrices (for either hierarchal or grouped time series), the result is a set of coherent forecasts. The challenge then is finding the "optimal" mapping matrix that can be used to reconcile the forecasts with the least amount of variance.

## Reconciliation Methods
Many different reconciliation methods exist, and some are better than others. I will not go into detail on the majority of them, but I will still list a few. Reconciliation methods fall under two categories: [single-level](https://otexts.com/fpp3/single-level.html) and [minimum trace](https://otexts.com/fpp3/reconciliation.html#the-mint-optimal-reconciliation-approach) methods. 

Some single-level methods include:
- Bottom-Up
- Top-Down
- Proportions of historical averages (method exists for proportions of historical actuals as well as historical forecasts)
- Middle-Out 

Some minimum trace methods include:
- Ordinary least squares (OLS)
- Weighted least squares (WLS) with variance scaling
- WLS with structrual scaling

For a more rigorous explanation of the different methods - along with a few additional optimal approaches - feel free to check out [this](https://robjhyndman.com/papers/mint.pdf) paper.

## Available Reconciliation Tools
As mentioned previously, there are a couple of existing tools/libraries that can apply the majority of reconciliation methods. The aforementioned Rob Hyndman helped to develop an R package called [`fabletools`](https://fabletools.tidyverts.org/reference/reconcile.html) that in my opinion is one of the best forecasting packages, and the best tool out there for forecast reconciliation. An effort is also underway to develop a Python package called [`scikit-hts`](https://scikit-hts.readthedocs.io/en/latest/readme.html). However, it is still pretty buggy and doesn't have all of the capabilities that exist in `fabletools`.

## Forecast Reconciliation in Python

In [10]:
import numpy as np

import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'src'))
from utils import get_sample_data
import hyperarch

# Read data
h_df = get_sample_data(verbose=False)
hierarchy_df, h_bottom, h_labels = hyperarch.get_hierarchal(h_df, 'category', 'subcategory', agg_type='hierarchy') 
h_s = hyperarch.get_S(h_bottom, h_labels, agg_type='hierarchy')
h_models = hyperarch.get_models(hierarchy_df, method='auto_arima', steps_out=6, verbose=False)
h_yhat = hyperarch.get_forecast_matrix(h_models)
h_reconciled_yhat = hyperarch.reconcile(h_yhat, h_s, method='ols')
hdf_pred, hdf_rec = hyperarch.predict_hier(hierarchy_df, h_yhat, h_reconciled_yhat, h_labels, h_models['index'])

In [ ]:
###TODO: OLS reconciliation using the hierarchy described in the first example -> use numpy, not pandas

## References
[1] Rob J. Hyndman and George Athanasopoulos, *Forecasting: Principles and Practice 3rd ed.* (2022), https://otexts.com/fpp3/ <br>
[2] Shanika Wickramasuriya, George Athanasopoulos and Rob Hyndman; *Optimal Forecast Reconciliation for Hierarchical and Grouped Time Series through Trace Minimization* (2017); https://robjhyndman.com/papers/mint.pdf <br>
[3] Mitchell O'Hara-Wild, Rob Hyndman, Earo Wang; *fabletools* (2022); https://fabletools.tidyverts.org/reference/reconcile.html <br>
[4] Carlo Mazzaferro, *scikit-hts* (2019), https://scikit-hts.readthedocs.io/en/latest/readme.html